In [402]:
%matplotlib inline

import math
import subprocess
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats.stats import pearsonr


In [403]:
def tra2rel(fileinput, fileoutput, delimiter=',', has_header=True):
    data = open(fileinput, 'r')
    if has_header:
        data.readline()
    baskets = defaultdict(list)

    for row in data:
        basket_id = row.replace('\r\n', '').split(delimiter)[0]
        item_id = row.replace('\r\n', '').split(delimiter)[1]
        baskets[basket_id].append(item_id)

    data.close()

    out = open(fileoutput, 'w')
    for k, v in baskets.iteritems():
        s = '%s' % k
        for item in v:
            s += ',%s' % item
        out.write('%s\n' % s)
    out.close()
    
    return baskets

In [404]:
def call_apriori(fileinput, fileoutput, delimiter=',', target_type='s', 
                 min_nbr_items=1, min_sup=2, min_conf=2):
    # apriori
    # -t# {m: maximal, c: closed, s: frequent, r: association rules}
    # -m# minimum number of items per item set/association rule
    # -s# minimum support of an item set, positive: percentage, negative: absolute
    # -c# minimum confidence rule percentage
    # -b# line delimiter (,)
    # The default additional information output format for rules is " (%X, %C)"
    # %X relative body set support as a percentage
    # %C rule confidence as a percentage
    # %L lift

    if target_type == 'r':
        call_cmd = ['./apriori', '-b%s' % delimiter, '-t%s' % target_type, '-m%s' % min_nbr_items, 
                    '-s%s' % min_sup, '-c%s' % min_conf, '-v (%X, %C, %L)', 
                    fileinput, fileoutput]
    else:
        call_cmd = ['./apriori', '-b%s' % delimiter, '-t%s' % target_type, 
                           '-m%s' % min_nbr_items, '-s%s' % min_sup, fileinput, fileoutput]

    ret = subprocess.call(call_cmd,  stdout=open('apriori_stdout.txt', 'w'), 
                          stderr=open('apriori_stderr.txt', 'w'))
    return ret

In [405]:
def read_rules(filename):
    data = open(filename, 'r')
    rules = list()
    for row in data:
        fileds = row.rstrip('\n\r').split(' <- ')
        cons = fileds[0]
        other = fileds[1].split(' (')
        ant = other[0].split(' ')
        other2 = other[1].split(', ')
        sup = float(other2[0])
        conf = float(other2[1])
        lift = float(other2[2].replace(')', ''))
        rule = {
            'ant': ant,
            'cons': cons,
            'sup': sup,
            'conf': conf,
            'lift': lift
        }
        rules.append(rule)
    data.close()
    return rules

In [406]:
# Frequent Pattern Mining on Titanic Dataset

In [407]:
df = pd.read_csv("training.csv")

In [408]:
def clean_data(df):
    df.replace({'Trim' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'SubModel' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'Color' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'Transmission' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'WheelTypeID' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'WheelType' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'Nationality' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'Size' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'TopThreeAmericanName' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRAcquisitionAuctionAveragePrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRAcquisitionAuctionCleanPrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRAcquisitionRetailAveragePrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRAcquisitonRetailCleanPrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRCurrentAuctionAveragePrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRCurrentAuctionCleanPrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRCurrentRetailAveragePrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRCurrentRetailCleanPrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'PRIMEUNIT' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'AUCGUART' : { np.nan : 'NS'}}, inplace=True)

    df['Transmission'] = df['Transmission'].str.upper()

    df = df.drop('PurchDate', axis=1)
    df = df.drop('VehYear',axis=1)
    df = df.drop('WheelTypeID', axis=1)
    df = df.drop('TopThreeAmericanName', axis=1)
    df = df.drop('BYRNO', axis=1)

    auctions = sorted(df['Auction'].unique())
    vehicleAges = sorted(df['VehicleAge'].unique())
    makes = sorted(df['Make'].unique())
    models = sorted(df['Model'].unique())
    trims = sorted(df['Trim'].unique())
    subModels = sorted(df['SubModel'].unique())
    colors = sorted(df['Color'].unique())
    transmissions = sorted(df['Transmission'].unique())
    wheeltypes = sorted(df['WheelType'].unique())
    vehOdos = sorted(df['VehOdo'].unique())
    nationalities = sorted(df['Nationality'].unique())
    sizes = sorted(df['Size'].unique())
    demands = sorted(df['PRIMEUNIT'].unique())
    guarantees = sorted(df['AUCGUART'].unique())
    zipcodes = sorted(df['VNZIP1'].unique())    #VNZIP1
    states = sorted(df['VNST'].unique())        #VNST
    vehcosts = sorted(df['VehBCost'].unique())
    onlinesales = sorted(df['IsOnlineSale'].unique())
    warranties = sorted(df['WarrantyCost'].unique())
    
    auctionsDictionary = dict(zip(auctions,range(0, len(auctions)+1)))
    makesDictionary = dict(zip(makes,range(0, len(makes)+1)))
    modelsDictionary = dict(zip(models,range(0, len(models)+1)))
    trimsDictionary = dict(zip(trims,range(0, len(trims)+1)))
    subModelsDictionary = dict(zip(subModels,range(0, len(subModels)+1)))
    colorsDictionary = dict(zip(colors,range(0, len(colors)+1)))
    transmissionsDictionary = dict(zip(transmissions,range(0, len(transmissions)+1)))
    wheeltypesDictionary = dict(zip(wheeltypes,range(0, len(wheeltypes)+1)))
    nationalitiesDictionary = dict(zip(nationalities,range(0, len(nationalities)+1)))
    sizesDictionary = dict(zip(sizes,range(0, len(sizes)+1)))
    demandsDictionary = dict(zip(demands,range(0, len(demands)+1)))
    guaranteesDictionary = dict(zip(guarantees,range(0, len(guarantees)+1)))
    vehcostsDictionary = dict(zip(vehcosts,range(0, len(vehcosts)+1)))  #not object but mapped as dictionary
    zipcodesDictionary = dict(zip(zipcodes,range(0, len(zipcodes)+1)))  #not object but mapped as dictionary
    statesDictionary = dict(zip(states,range(0, len(states)+1)))
    warrantiesDictionary = dict(zip(warranties,range(0, len(warranties)+1)))
    
    df['AuctionVal'] = df['Auction'].map(auctionsDictionary)
    df['MakeVal'] = df['Make'].map(makesDictionary)
    df['ModelVal'] = df['Model'].map(modelsDictionary)
    df['TrimVal'] = df['Trim'].map(trimsDictionary)
    df['SubModelVal'] = df['SubModel'].map(subModelsDictionary)
    df['ColorVal'] = df['Color'].map(colorsDictionary)
    df['TransmissionVal'] = df['Transmission'].map(transmissionsDictionary)
    df['WheelTypeVal'] = df['WheelType'].map(wheeltypesDictionary)
    df['NationalityVal'] = df['Nationality'].map(nationalitiesDictionary)
    df['SizeVal'] = df['Size'].map(sizesDictionary)
    df['PRIMEUNITVal'] = df['PRIMEUNIT'].map(demandsDictionary)
    df['AUCGUARTVal'] = df['AUCGUART'].map(guaranteesDictionary)
    df['VehBCostVal'] = df['VehBCost'].map(vehcostsDictionary)
    df['VNZIP1Val'] = df['VNZIP1'].map(zipcodesDictionary)
    df['VNSTVal'] = df['VNST'].map(statesDictionary)
    df['WarrantyCostVal'] = df['WarrantyCost'].map(warrantiesDictionary)

    dfFeatureSelection = df.drop(['RefId','MMRAcquisitionAuctionAveragePrice','MMRAcquisitionAuctionCleanPrice','MMRAcquisitionRetailAveragePrice',
                                 'MMRAcquisitonRetailCleanPrice','MMRCurrentAuctionAveragePrice','MMRCurrentAuctionCleanPrice','MMRCurrentRetailAveragePrice',
                                 'MMRCurrentRetailCleanPrice'], axis=1)

    #dfFeatureSelection = dfFeatureSelection.drop(['Auction','Make','Model','Trim','SubModel','Color','Transmission','WheelType','Nationality','Size',
    #                              'PRIMEUNIT','AUCGUART','VehBCost','VNZIP1','VNST','WarrantyCost'], axis=1)

    nAttributesRanking = len(dfFeatureSelection.columns)
    
    return df

In [409]:
df2 = clean_data(df)

df2 = pd.concat([
    df2['IsBadBuy'],     # mah?! e' il target
    df2['VehicleAge'],
    df2['VehOdo'],
    df2['IsOnlineSale'],
    df2['Auction'],
    df2['Make'],
    df2['Model'],
    df2['Trim'],
    df2['SubModel'],
    df2['Color'],
    df2['Transmission'],
    df2['WheelType'],
    df2['Nationality'],
    df2['Size'],
    df2['PRIMEUNIT'],
    df2['AUCGUART'],
    df2['VehBCost'],
    df2['VNZIP1'],
    df2['VNST'],
    df2['WarrantyCost']
    ],
    axis=1)

#df2 = pd.concat([
#    df2['Make'],
    #df2['Model'],
    #df2['Trim'],
    #df2['SubModel'],
    #df2['Color'],
    #df2['Transmission'],
    #df2['WheelType'],
    #df2['Size'],
    #],
    #axis=1)

In [410]:
df2['Make'] = df2['Make'].map(lambda x: x.strip().replace(" ", "-"))
df2['Model'] = df2['Model'].map(lambda x: x.strip().replace(" ", "-"))
df2['Trim'] = df2['Trim'].map(lambda x: x.strip().replace(" ", "-"))
df2['SubModel'] = df2['SubModel'].map(lambda x: x.strip().replace(" ", "-"))
df2['Nationality'] = df2['Nationality'].map(lambda x: x.strip().replace(" ", "-"))
df2['Size'] = df2['Size'].map(lambda x: x.strip().replace(" ", "-"))
df2.head()

,IsBadBuy,VehicleAge,VehOdo,IsOnlineSale,Auction,Make,Model,Trim,SubModel,Color,Transmission,WheelType,Nationality,Size,PRIMEUNIT,AUCGUART,VehBCost,VNZIP1,VNST,WarrantyCost
0,0,3,89046,0,ADESA,MAZDA,MAZDA3,i,4D-SEDAN-I,RED,AUTO,Alloy,OTHER-ASIAN,MEDIUM,NS,NS,7100.0,33619,FL,1113
1,0,5,93593,0,ADESA,DODGE,1500-RAM-PICKUP-2WD,ST,QUAD-CAB-4.7L-SLT,WHITE,AUTO,Alloy,AMERICAN,LARGE-TRUCK,NS,NS,7600.0,33619,FL,1053
2,0,4,73807,0,ADESA,DODGE,STRATUS-V6,SXT,4D-SEDAN-SXT-FFV,MAROON,AUTO,Covers,AMERICAN,MEDIUM,NS,NS,4900.0,33619,FL,1389
3,0,5,65617,0,ADESA,DODGE,NEON,SXT,4D-SEDAN,SILVER,AUTO,Alloy,AMERICAN,COMPACT,NS,NS,4100.0,33619,FL,630
4,0,4,69367,0,ADESA,FORD,FOCUS,ZX3,2D-COUPE-ZX3,SILVER,MANUAL,Covers,AMERICAN,COMPACT,NS,NS,4000.0,33619,FL,1020


In [411]:
df2['VehOdo'] = pd.cut(df2['VehOdo'], bins=range(0, 100005, 10000), right=False, labels=range(0, 100000, 10000))

In [412]:
df2.head()

,IsBadBuy,VehicleAge,VehOdo,IsOnlineSale,Auction,Make,Model,Trim,SubModel,Color,Transmission,WheelType,Nationality,Size,PRIMEUNIT,AUCGUART,VehBCost,VNZIP1,VNST,WarrantyCost
0,0,3,80000,0,ADESA,MAZDA,MAZDA3,i,4D-SEDAN-I,RED,AUTO,Alloy,OTHER-ASIAN,MEDIUM,NS,NS,7100.0,33619,FL,1113
1,0,5,90000,0,ADESA,DODGE,1500-RAM-PICKUP-2WD,ST,QUAD-CAB-4.7L-SLT,WHITE,AUTO,Alloy,AMERICAN,LARGE-TRUCK,NS,NS,7600.0,33619,FL,1053
2,0,4,70000,0,ADESA,DODGE,STRATUS-V6,SXT,4D-SEDAN-SXT-FFV,MAROON,AUTO,Covers,AMERICAN,MEDIUM,NS,NS,4900.0,33619,FL,1389
3,0,5,60000,0,ADESA,DODGE,NEON,SXT,4D-SEDAN,SILVER,AUTO,Alloy,AMERICAN,COMPACT,NS,NS,4100.0,33619,FL,630
4,0,4,60000,0,ADESA,FORD,FOCUS,ZX3,2D-COUPE-ZX3,SILVER,MANUAL,Covers,AMERICAN,COMPACT,NS,NS,4000.0,33619,FL,1020


In [413]:
df2['WarrantyCost'] = pd.cut(df2['WarrantyCost'], bins=range(0, 3005, 500), right=False, labels=range(0, 3000, 500))

In [414]:
#df2.drop(['VehOdo', 'WarrantyCost', 'VehicleAge'], axis=1, inplace=True)
df2.head()

,IsBadBuy,VehicleAge,VehOdo,IsOnlineSale,Auction,Make,Model,Trim,SubModel,Color,Transmission,WheelType,Nationality,Size,PRIMEUNIT,AUCGUART,VehBCost,VNZIP1,VNST,WarrantyCost
0,0,3,80000,0,ADESA,MAZDA,MAZDA3,i,4D-SEDAN-I,RED,AUTO,Alloy,OTHER-ASIAN,MEDIUM,NS,NS,7100.0,33619,FL,1000
1,0,5,90000,0,ADESA,DODGE,1500-RAM-PICKUP-2WD,ST,QUAD-CAB-4.7L-SLT,WHITE,AUTO,Alloy,AMERICAN,LARGE-TRUCK,NS,NS,7600.0,33619,FL,1000
2,0,4,70000,0,ADESA,DODGE,STRATUS-V6,SXT,4D-SEDAN-SXT-FFV,MAROON,AUTO,Covers,AMERICAN,MEDIUM,NS,NS,4900.0,33619,FL,1000
3,0,5,60000,0,ADESA,DODGE,NEON,SXT,4D-SEDAN,SILVER,AUTO,Alloy,AMERICAN,COMPACT,NS,NS,4100.0,33619,FL,500
4,0,4,60000,0,ADESA,FORD,FOCUS,ZX3,2D-COUPE-ZX3,SILVER,MANUAL,Covers,AMERICAN,COMPACT,NS,NS,4000.0,33619,FL,1000


In [415]:
df3=df2
df3['IsBadBuy'] = df2['IsBadBuy'].astype(str) + '_IsBadBuy'
df3['VehicleAge'] = df2['VehicleAge'].astype(str) + '_VehicleAge'
df3['VehOdo'] = df2['VehOdo'].astype(str) + '_Odometer'
df3['IsOnlineSale'] = df2['IsOnlineSale'].astype(str) + '_IsOnlineSale'
df3['Auction'] = df2['Auction'].astype(str) + '_Auction'
df3['Make'] = df2['Make'].astype(str) + '_Make'
df3['Model'] = df2['Model'].astype(str) + '_Model'
df3['Trim'] = df2['Trim'].astype(str) + '_Trim'
df3['SubModel'] = df2['SubModel'].astype(str) + '_SubModel'
df3['Color'] = df2['Color'].astype(str) + '_Color'
df3['Transmission'] = df2['Transmission'].astype(str) + '_Transmission'
df3['WheelType'] = df2['WheelType'].astype(str) + '_WheelType'
df3['Nationality'] = df2['Nationality'].astype(str) + '_Nationality'
df3['Size'] = df2['Size'].astype(str) + '_Size'
df3['PRIMEUNIT'] = df2['PRIMEUNIT'].astype(str) + '_PRIMEUNIT'
df3['AUCGUART'] = df2['AUCGUART'].astype(str) + '_AUCGUART'
df3['VehBCost'] = df2['VehBCost'].astype(str) + '_VehBCost'
df3['VNZIP1'] = df2['VNZIP1'].astype(str) + '_VNZIP1'
df3['VNST'] = df2['VNST'].astype(str) + '_VNST'
df3['WarrantyCost'] = df2['WarrantyCost'].astype(str) + '_WarrantyCost'

In [416]:
df3.drop(['IsOnlineSale','AUCGUART', 'PRIMEUNIT', 'Transmission'], axis=1, inplace=True)

In [417]:
df3.head()

,IsBadBuy,VehicleAge,VehOdo,Auction,Make,Model,Trim,SubModel,Color,WheelType,Nationality,Size,VehBCost,VNZIP1,VNST,WarrantyCost
0,0_IsBadBuy,3_VehicleAge,80000.0_Odometer,ADESA_Auction,MAZDA_Make,MAZDA3_Model,i_Trim,4D-SEDAN-I_SubModel,RED_Color,Alloy_WheelType,OTHER-ASIAN_Nationality,MEDIUM_Size,7100.0_VehBCost,33619_VNZIP1,FL_VNST,1000.0_WarrantyCost
1,0_IsBadBuy,5_VehicleAge,90000.0_Odometer,ADESA_Auction,DODGE_Make,1500-RAM-PICKUP-2WD_Model,ST_Trim,QUAD-CAB-4.7L-SLT_SubModel,WHITE_Color,Alloy_WheelType,AMERICAN_Nationality,LARGE-TRUCK_Size,7600.0_VehBCost,33619_VNZIP1,FL_VNST,1000.0_WarrantyCost
2,0_IsBadBuy,4_VehicleAge,70000.0_Odometer,ADESA_Auction,DODGE_Make,STRATUS-V6_Model,SXT_Trim,4D-SEDAN-SXT-FFV_SubModel,MAROON_Color,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,4900.0_VehBCost,33619_VNZIP1,FL_VNST,1000.0_WarrantyCost
3,0_IsBadBuy,5_VehicleAge,60000.0_Odometer,ADESA_Auction,DODGE_Make,NEON_Model,SXT_Trim,4D-SEDAN_SubModel,SILVER_Color,Alloy_WheelType,AMERICAN_Nationality,COMPACT_Size,4100.0_VehBCost,33619_VNZIP1,FL_VNST,500.0_WarrantyCost
4,0_IsBadBuy,4_VehicleAge,60000.0_Odometer,ADESA_Auction,FORD_Make,FOCUS_Model,ZX3_Trim,2D-COUPE-ZX3_SubModel,SILVER_Color,Covers_WheelType,AMERICAN_Nationality,COMPACT_Size,4000.0_VehBCost,33619_VNZIP1,FL_VNST,1000.0_WarrantyCost


In [418]:
df3.to_csv('for_pattern.csv', header=False)

In [419]:
delimiter=','
target_type='s' # s, c, m, g
min_nbr_items=3
min_sup=5
min_conf=0

ret_val = call_apriori('for_pattern.csv', 'patterns.txt', 
                       delimiter, target_type, min_nbr_items, min_sup, min_conf)

In [429]:
delimiter=','
target_type='r'
min_nbr_items=2
min_sup=5
min_conf=16

ret_val = call_apriori('for_pattern.csv', 'rules.txt', delimiter, target_type, 
                       min_nbr_items, min_sup, min_conf)

In [430]:
rules = read_rules('rules.txt')
for r in rules: #[:10000]:
    print r['ant'], '-->', r['cons'], ' lift', r['lift'], ' conf', r['conf']

['TOP-LINE-ASIAN_Nationality'] --> ALTIMA_Model  lift 1960.85  conf 18.2966
['CHEVROLET_Make', 'MEDIUM_Size', 'AMERICAN_Nationality', '0_IsBadBuy'] --> MALIBU-4C-2.2L-I4-MP_Model  lift 1767.49  conf 17.3158
['CHEVROLET_Make', 'MEDIUM_Size', 'AMERICAN_Nationality'] --> MALIBU-4C-2.2L-I4-MP_Model  lift 1756.51  conf 17.2082
['CHEVROLET_Make', 'MEDIUM_Size', '0_IsBadBuy'] --> MALIBU-4C-2.2L-I4-MP_Model  lift 1767.49  conf 17.3158
['CHEVROLET_Make', 'MEDIUM_Size'] --> MALIBU-4C-2.2L-I4-MP_Model  lift 1756.51  conf 17.2082
['LARGE_Size', 'CHEVROLET_Make', 'AMERICAN_Nationality', '0_IsBadBuy'] --> IMPALA-3.5L-V6-SFI_Model  lift 1480.45  conf 16.7756
['LARGE_Size', 'CHEVROLET_Make', 'AMERICAN_Nationality'] --> IMPALA-3.5L-V6-SFI_Model  lift 1424.06  conf 16.1366
['LARGE_Size', 'CHEVROLET_Make', '0_IsBadBuy'] --> IMPALA-3.5L-V6-SFI_Model  lift 1480.45  conf 16.7756
['LARGE_Size', 'CHEVROLET_Make'] --> IMPALA-3.5L-V6-SFI_Model  lift 1424.06  conf 16.1366
['LARGE_Size', 'Covers_WheelType', 'AMER

In [422]:
rulse_cons_S = list()
for r in rules:
    if r['cons'].endswith('_S'):
        rulse_cons_S.append(r)

In [423]:
print len(rulse_cons_S)

0


In [424]:
sorted_rules_cons_S = sorted(rulse_cons_S, key=lambda r: r['conf'], reverse=True)

In [425]:
for r in sorted_rules_cons_S[:10]:
    print r['ant'], '-->', r['cons'], ' lift', r['lift'], ' conf', r['conf']

In [426]:
df3.values[0]

array(['0_IsBadBuy', '3_VehicleAge', '80000.0_Odometer', 'ADESA_Auction',
       'MAZDA_Make', 'MAZDA3_Model', 'i_Trim', '4D-SEDAN-I_SubModel',
       'RED_Color', 'Alloy_WheelType', 'OTHER-ASIAN_Nationality',
       'MEDIUM_Size', '7100.0_VehBCost', '33619_VNZIP1', 'FL_VNST',
       '1000.0_WarrantyCost'], dtype=object)

In [427]:
passenger_test = df3.values[0]

In [428]:
for r in rules:
    if set(r['ant']) < set(passenger_test):
        print r['ant'], '-->', r['cons']

['FL_VNST', 'MEDIUM_Size'] --> 0_IsBadBuy
['ADESA_Auction', '1000.0_WarrantyCost', '0_IsBadBuy'] --> AMERICAN_Nationality
['ADESA_Auction', '1000.0_WarrantyCost'] --> AMERICAN_Nationality
['ADESA_Auction', 'Alloy_WheelType'] --> 0_IsBadBuy
['3_VehicleAge', '1000.0_WarrantyCost', '0_IsBadBuy'] --> AMERICAN_Nationality
['3_VehicleAge', '1000.0_WarrantyCost'] --> AMERICAN_Nationality
['3_VehicleAge', '1000.0_WarrantyCost'] --> 0_IsBadBuy
['3_VehicleAge', 'MEDIUM_Size'] --> 0_IsBadBuy
['3_VehicleAge', 'Alloy_WheelType'] --> 0_IsBadBuy
['3_VehicleAge'] --> 0_IsBadBuy
['1000.0_WarrantyCost', 'MEDIUM_Size', 'Alloy_WheelType', '0_IsBadBuy'] --> AMERICAN_Nationality
['1000.0_WarrantyCost', 'MEDIUM_Size', 'Alloy_WheelType'] --> AMERICAN_Nationality
['1000.0_WarrantyCost', 'MEDIUM_Size', '0_IsBadBuy'] --> AMERICAN_Nationality
['1000.0_WarrantyCost', 'MEDIUM_Size'] --> AMERICAN_Nationality
['1000.0_WarrantyCost', '0_IsBadBuy'] --> AMERICAN_Nationality
['1000.0_WarrantyCost'] --> AMERICAN_Nationali